# QLoRA(Quantized Low-Rank Adapter)

### 1-Installing Required Librairies

In [1]:
!pip install transformers datasets peft bitsandbytes accelerate evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 20.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.5 MB/s eta 0:00:00


### 2- Importing Dependencies 

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
import evaluate
import torch

### 3. Loading and Splitting the Dataset

In [3]:
dataset = load_dataset("ag_news")

train_dataset = dataset["train"].shuffle(seed=42)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size

from datasets import DatasetDict
dataset = DatasetDict({
    "train": train_dataset.select(range(train_size)),
    "validation": train_dataset.select(range(train_size, train_size + val_size)),
    "test": dataset["test"]
})

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

### 4. Preprocessing Data

In [4]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

dataset = dataset.map(preprocess, batched=True).rename_column("label", "labels")
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/96000 [00:00<?, ? examples/s]

Map:   0%|          | 0/24000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

### 5. Loading Quantized Base Model for QLoRA

In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=4,  
    device_map="auto",
    quantization_config=bnb_config
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 6. Applying LoRA Adapters

In [6]:
qlora_adapter_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)

model = get_peft_model(model, qlora_adapter_config)

### 7. Training the Model

In [7]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return metric.compute(predictions=preds, references=labels)

qlora_training_args = TrainingArguments(
    output_dir="./qlora_agnews",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    num_train_epochs=2,
    eval_strategy="epoch",
    logging_steps=50,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=qlora_training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_metrics
)

trainer.train()
model.save_pretrained("./qlora_agnews_adapter")
print("QLoRA fine-tuning complete! Adapter saved at ./qlora_agnews_adapter")

Epoch,Training Loss,Validation Loss,Accuracy
1,0.220900,0.284424,0.912583
2,0.258900,0.262939,0.921542


QLoRA fine-tuning complete! Adapter saved at ./qlora_agnews_adapter


### 8. Testing the Model

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

class_labels = dataset["train"].features["labels"].names

samples = [
    "The stock market closed at an all-time high today.",
    "The football team won the championship after a thrilling game.",
    "Scientists discovered a new exoplanet in the habitable zone.",
    "The president met with foreign leaders to discuss trade agreements."
]

inputs = tokenizer(samples, truncation=True, padding=True, return_tensors="pt").to(device)

model.eval()
with torch.no_grad():
    logits = model(**inputs).logits
    preds = torch.argmax(logits, dim=-1)

for text, pred in zip(samples, preds):
    print(f"News: {text}\nPredicted Topic: {class_labels[pred]}\n")

News: The stock market closed at an all-time high today.
Predicted Topic: Business

News: The football team won the championship after a thrilling game.
Predicted Topic: Sports

News: Scientists discovered a new exoplanet in the habitable zone.
Predicted Topic: Sci/Tech

News: The president met with foreign leaders to discuss trade agreements.
Predicted Topic: Business

